In [ ]:
from datetime import datetime
import praw


from Author import Author


from document_factory import DocumentFactory


import urllib, urllib.request


import xmltodict
import dill

In [ ]:
listData = []



def fetch_reddit_data(query, limit):

    reddit = praw.Reddit(

        client_id="UKtZgFhv6izOz1HVZYi4iQ",

        client_secret="DKMUyLlfc-6pjWK2XE4fNSN6O7C5nA",

        user_agent="Test App",
    )

    hot_posts = reddit.subreddit(query).hot(limit=limit)

    for post in hot_posts:

        if post.selftext != "":
            titre = (

                post.title.encode("utf-8", errors="ignore")

                .decode("utf-8")

                .replace("\n", "")
            )

            auteur = str(post.author)

            date = datetime.fromtimestamp(post.created_utc).strftime(
                "%Y/%m/%d"
            )
            url = post.url

            texte = post.selftext.replace("\n", "")

            doc = DocumentFactory.create_document(

                "Reddit",

                titre,

                auteur,

                date,

                url,

                texte,
            )

        listData.append(("Reddit", doc))



def fetch_arxiv_data(query, limit):
    url = (

        "http://export.arxiv.org/api/query?search_query=all:"

        + query

        + "&start=0&max_results="
        + str(limit)
    )

    data = urllib.request.urlopen(url).read().decode("utf-8")

    # transform the data from xml to json


    dataToJson = xmltodict.parse(data)


    # get the specific data that i need

    adocs = dataToJson["feed"]["entry"]


    for aPost in adocs:

        titre = aPost["title"].replace("\n", "")

        try:

            authors = ", ".join(

                [a["name"] for a in aPost["author"]]

            )  # On fait une liste d'auteurs, séparés par une virgule

        except:

            authors = aPost["author"][
                "name"
            ]  # Si l'auteur est seul, pas besoin de liste


        summary = aPost["summary"].replace("\n", "")  # On enlève les retours à la ligne

        date = datetime.strptime(aPost["published"], "%Y-%m-%dT%H:%M:%SZ").strftime(

            "%Y/%m/%d"

        )  # Formatage de la date en année/mois/jour avec librairie datetime


        adoc = DocumentFactory.create_document(

            "Arxiv",

            titre,

            authors,

            date,

            aPost["link"][1]["@href"],

            summary,
        )

        listData.append(("Arxiv", adoc))



id2doc = {}



def fillDocDict():

    counter = 0
    for origin, data in listData:
        id = f"{origin}_{counter}"
        id2doc[id] = data
        counter += 1


id2aut = {}
authors = {}


def fillAuthorsDict():
    idAuthor = 0
    for doc in id2doc.values():
        if doc.auteur not in id2aut:
            idAuthor += 1
            authors[idAuthor] = Author(doc.auteur)
            id2aut[doc.auteur] = authors[idAuthor]


from Corpus import Corpus

CorpusNoSingleton = Corpus
del Corpus
corpus = CorpusNoSingleton("Mon corpus")

nbPosts = 0


def fillCorpus(docType=""):
    global nbPosts
    for origin, d in listData:
        if docType == origin:
            corpus.add(d)
            nbPosts += 1
        else:
            corpus.add(d)
            nbPosts += 1







In [ ]:
import ipywidgets as widgets
from IPython.display import display
from decorators import singleton


corpus = None  # Initialisation de la variable corpus


@singleton
class Corpus(CorpusNoSingleton):
    pass


mot_clef_textbox = widgets.Text(description="Mot-clé :")
nb_articles_slider = widgets.IntSlider(
    description="Nombre d'articles :", min=1, max=100, value=10
)



# Affichage des champs du formulaire


display(mot_clef_textbox)


display(nb_articles_slider)



# Fonction pour récupérer les données lorsque l'utilisateur valide le formulaire


def on_submit_button_clicked(button):
    global corpus
    corpus = Corpus("Mon corpus")

    query = mot_clef_textbox.value

    limit = nb_articles_slider.value
    listData = []
    # Utilisation des valeurs saisies par l'utilisateur pour récupérer les données des deux APIs

    fetch_reddit_data(query, limit // 2)

    fetch_arxiv_data(query, limit // 2)
    fillDocDict()
    fillAuthorsDict()


    fillCorpus()
    with open(str(query) + "_" + "corpus.pkl", "wb") as f:
        dill.dump(corpus, f)

    with open(str(query) + "_" + "corpus.pkl", "rb") as f:
        corpus = dill.load(f)
    print("Nombre total de posts dans le corpus :", nbPosts)
    print(corpus)



# Création du bouton de soumission du formulaire


submit_button = widgets.Button(description="Valider")


submit_button.on_click(on_submit_button_clicked)


display(submit_button)